<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://drive.google.com/uc?export=view&id=1LqkEgbpZj8A99Y9T59eBp9fEIZbpg6P2" alt="Snowflake Snowpark Classroom" style="width: 800px">
</div>

# Classroom 1.6 - Understanding Stored Procedures and User Defined Functions with Snowflake Snowpark for Python

In this notebook, you will learn the basics on how to create snowflake's stored procedures and user defined functions using Snowpark API for Python

## Learning Objectives

By the end of this classroom, you should be able to:
- Understanding Snowflake Stored Procedure Creation using Snowpark for Python
- Understanding Snowflake User Defined Functions Creation using Snowpark for Python

In [1]:
# Let's get started with Snowflake Snowpark for Python
from assets.config import connection_builder
session = connection_builder()

c:\Users\divya\anaconda3\envs\py38_env\lib\site-packages\snowflake\connector\options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


## Section 1 - Snowflake Stored Procedures

snowflake.snowpark.stored_procedure.StoredProcedure encapsulates a user defined lambda or function that is returned by [sproc()](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.functions.sproc.html#snowflake.snowpark.functions.sproc), [StoredProcedureRegistration.register()](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register.html#snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register) or [StoredProcedureRegistration.register_from_file()](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register_from_file.html#snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register_from_file). The constructor of this class is not supposed to be called directly.


### StoredProcedureRegistration 

snowflake.snowpark.stored_procedure.StoredProcedureRegistration provides methods to register lambdas and functions as stored procedures in the Snowflake database. 

- ***session.sproc*** returns an object of this class. You can use this object to register stored procedures that you plan to use in the current session or permanently. The methods that register a stored procedure return a StoredProcedure object.
- **The first parameter of your function should be a snowpark Session**. Also, you need to add snowflake-snowpark-python package (version >= 0.4.0) to your session before trying to create a stored procedure.

#### Ways to Register Stored Procedure

- Use sproc() or [register()](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register#snowflake.snowpark.stored_procedure.StoredProcedureRegistration.register).
- Use register_from_file().

Additional Reference Codes for registering stored procedures using Snowflake snowpark API - [here](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.stored_procedure.StoredProcedureRegistration)


---------------------------------------

### Snowflake Snowpark supported data types for the parameters

Snowflake supports the following [data types](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.stored_procedure.StoredProcedureRegistration) for the parameters for a stored procedure:


| Python Type | Snowpark Type | SQL Type | 
|--|--|--|
| int |  LongType | NUMBER | 
| decimal.Decimal | DecimalType | NUMBER |
| float | FloatType | FLOAT |
| str | StringType | STRING |
| bool | BooleanType | BOOL |
| datetime.time | TimeType | TIME |
| datetime.date | DateType | DATE |
| datetime.datetime | TimestampType | TIMESTAMP |
| bytes or bytearray | BinaryType | BINARY |
| list | ArrayType | ARRAY |
| dict | MapType | OBJECT |
| Dynamically mapped to the native Python type | VariantType | VARIANT |
| dict | GeographyType | GEOGRAPHY |


In [2]:
import snowflake.snowpark
from snowflake.snowpark.functions import sproc, col
from snowflake.snowpark.types import StringType, BooleanType, IntegerType
session.add_packages('snowflake-snowpark-python')
session.sql('CREATE OR REPLACE STAGE TEST_STAGE').collect()

print('----------------- Query History Table Refresh SP Creation Using @Sproc() Decorator -----------------')
# @sproc decorator allows you to create temporary stored procedure which you can use within the session
@sproc
def refresh_query_history(session: snowflake.snowpark.Session)-> str:
    tgt_table = session.sql('CREATE TABLE IF NOT EXISTS Q_HISTORY LIKE SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY').collect()
    query_history_table = session.table('SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY')
    query_history_table.write.mode('overwrite').save_as_table('Q_HISTORY')
    return 'success'
refresh_query_history()
session.table('Q_HISTORY').sort(col('start_time').desc()).show()

print('----------------- Login History Table Refresh SP Creation Using session.sproc.register -----------------')
# Registering Stored Procedure using Session.sproc.register() method. It allows you to create permanent as well as temporary sprocs.
# For permanent stored procedures, you would require a stage to be mentioned while calling the method

def refresh_login_history(session : snowflake.snowpark.Session, status:bool)->str:
    if(status):
        login_history_table = session.table('SNOWFLAKE.ACCOUNT_USAGE.LOGIN_HISTORY')
        login_history_table.write.mode('overwrite').save_as_table('L_HISTORY')
        return 'refresh ran successfully'
    return 'table not refreshed'
session.sproc.register(name='refresh_login_hist_sp',
                       func=refresh_login_history,
                       return_type=StringType(),
                       input_types=[BooleanType()],
                       replace=True,
                       is_permanent=True,
                       stage_location='@TEST_STAGE',
                       execute_as = 'CALLER'
)
session.call('refresh_login_hist_sp',True)

print('----------------- 5+10 Sum Using Stored procedure from External File Using session.sproc.register -----------------')
# Registering Stored Procedure using Session.sproc.register_from_file() method. It allows you to register a python function from an external file or from a stage file

session.sproc.register_from_file(file_path='assets/misc.py',
                                 func_name='sum_val',
                                 return_type=IntegerType(),
                                 name='sum_values',
                                 is_permanent=True,
                                 stage_location='@TEST_STAGE',
                                 replace=True,
                                 )
session.call('sum_values')

----------------- Query History Table Refresh SP Creation Using @Sproc() Decorator -----------------
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

15

## Section 2 - Snowflake Snowpark User Defined Functions

snowflake.snowpark.udf.UDFRegistration provides methods to register lambdas and functions as UDFs in the Snowflake database.

[session.udf](https://docs.snowflake.com/developer-guide/snowpark/reference/python/api/snowflake.snowpark.Session.udf.html#snowflake.snowpark.Session.udf) returns an object of this class. You can use this object to register UDFs that you plan to use in the current session or permanently. The methods that register a UDF return a UserDefinedFunction object, which you can also use in Column expressions.

Before creating a UDF, think about whether you want to create a vectorized UDF (also referred to as Python UDF Batch API) or a regular UDF. The advantages of a vectorized UDF are:
- The potential for better performance if your Python code operates efficiently on batches of rows.
- Less transformation logic is required if you are calling into libraries that operate on Pandas DataFrames or Pandas arrays.

#### Ways to Register User Defined Stored Procedure

- Use [udf()](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.functions.udf#snowflake.snowpark.functions.udf) or [register()](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.udf.UDFRegistration.register#snowflake.snowpark.udf.UDFRegistration.register).
- Use [register_from_file()](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.udf.UDFRegistration.register_from_file#snowflake.snowpark.udf.UDFRegistration.register_from_file).

Additional Reference Codes for registering UDFs using Snowflake snowpark API - [here](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.udf.UDFRegistration)



In [3]:
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import StructField, StructType, StringType, IntegerType, FloatType
from assets.src_udfs import credit_to_dollar

schema = StructType([StructField('warehouse',StringType()),StructField('credit',IntegerType())])
dataset = [['XS',23],['L',66],['M',11]]
df = session.createDataFrame(dataset,schema= schema)



print('----------------- Credit Consumption UDF Example using @udf() Decorator -----------------')
# @udf decorator allows you to create temporary UDFs which you can use within the session

@udf(imports=[('assets/src_udfs.py','assets.src_udfs')])
def wh_credit_calculator(wh_size:str, credit:int)->float:
    
    if wh_size == 'XS':
        return credit_to_dollar(credit)*1
    elif  wh_size == 'S':
        return credit_to_dollar(credit)*2
    elif  wh_size == 'M':
        return credit_to_dollar(credit)*4
    elif  wh_size == 'L':
        return credit_to_dollar(credit)*8
    elif  wh_size == 'XL':
        return credit_to_dollar(credit)*16
    elif  wh_size == 'XXL':
        return credit_to_dollar(credit)*32
    else:
        return credit_to_dollar(credit)

df.select(wh_credit_calculator('WAREHOUSE','CREDIT')).show()


print('----------------- Credit To Dollar UDF Example using session.udf.register_from_file() method -----------------')
# Registering UDF using Session.udf.register_from_file() method. It allows you to register a python function from an external file or from a stage file


c_to_d = session.udf.register_from_file(file_path = 'assets/src_udfs.py',
                                        func_name ='credit_to_dollar',
                                        replace=True,
                                        name = 'c_to_d',
                                        return_type = FloatType(),
                                        input_types = [IntegerType()],
                                        is_permanent = True,
                                        stage_location='@TEST_STAGE',
                            )

df.select(c_to_d(df.credit)).show()

----------------- Credit Consumption UDF Example using @udf() Decorator -----------------
------------------------------------------------------
|"""UDEMY"".""PUBLIC"".SNOWPARK_TEMP_FUNCTION_0P...  |
------------------------------------------------------
|48.3                                                |
|92.4                                                |
|1108.8                                              |
------------------------------------------------------

----------------- Credit To Dollar UDF Example using session.udf.register_from_file() method -----------------
------------------------
|"C_TO_D(""CREDIT"")"  |
------------------------
|48.3                  |
|23.1                  |
|138.6                 |
------------------------

